# Week 10 — NLP & Generative AI (Intro)

**Goals**
- Classic NLP: tokenization, TF‑IDF + classifier
- GenAI: use a Transformers pipeline for sentiment
- Simple prompt-engineering ideas; small offline RAG-like demo with TF‑IDF

## 0) Setup

In [ ]:
# !pip -q install pandas scikit-learn matplotlib seaborn transformers
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
print('Setup ready')

## 1) Classic NLP — TF‑IDF + Logistic Regression

In [ ]:
texts = [
    'I loved the movie, fantastic acting!', 'Worst film ever, boring and slow.',
    'Amazing soundtrack and visuals', 'Not my taste, would not recommend',
    'Great plot and characters', 'Terrible script and awful pacing',
    'Absolutely brilliant!', 'It was okay, not great', 'Bad effects and story', 'Really enjoyed it!'
]
labels = [1,0,1,0,1,0,1,0,0,1]  # 1=pos, 0=neg
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer(stop_words='english')
Xtr = tfidf.fit_transform(X_train); Xte = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=200).fit(Xtr, y_train)
pred = clf.predict(Xte)
print(classification_report(y_test, pred))

## 2) GenAI — Transformers sentiment pipeline

In [ ]:
from transformers import pipeline
# This will download a small default model (requires internet on first run)
sentiment = pipeline('sentiment-analysis')
sentiment(['I absolutely love this course!', 'This was a waste of time.'])

## 3) Simple RAG‑like retrieval (TF‑IDF)

In [ ]:
corpus = [
    'ETL pipelines move and transform data between systems.',
    'Orchestration tools like Prefect schedule and monitor workflows.',
    'A data warehouse organizes facts and dimensions for BI.',
    'Supervised ML uses labeled data for prediction.',
    'Generative AI can summarize, translate, and generate content.'
]
vectorizer = TfidfVectorizer().fit(corpus)
doc_mat = vectorizer.transform(corpus)

def retrieve(query, topk=2):
    q = vectorizer.transform([query])
    scores = (doc_mat @ q.T).toarray().ravel()
    idx = scores.argsort()[::-1][:topk]
    return [(corpus[i], float(scores[i])) for i in idx]

retrieve('How do we schedule data workflows?')

## 4) Prompt tips (write-ups)

- Provide **role + task + constraints** (e.g., “You are a data engineer. Draft an Airflow DAG.”)

- Use **few-shot examples** to steer style/format.

- Ask for **chain-of-thought summaries** and **verification steps** (when appropriate for learning; avoid leaking private reasoning in production).